In [5]:
import pandas as pd
import numpy as np
import datetime

код услуги 0522.006220

In [6]:
doctor = pd.read_excel('Акименко.xlsx', dtype={'Номер карты' : str})
pacients = pd.read_excel('ПЦР.xlsx', dtype={'Номер карты' : str})

In [7]:
pacients['Период_действия'] = pacients['Дата приема'].apply(lambda x: pd.date_range(start=x.date() - pd.Timedelta(days=14), end=x.date() + pd.Timedelta(days=14))).apply(lambda x: x.to_list()).apply(lambda x: [i.date() for i in x])

In [8]:
group_pacients = pacients.groupby('Номер карты', as_index=False).agg({'Период_действия' : sum})

In [9]:
doctor = doctor.merge(group_pacients, left_on='Номер карты', right_on='Номер карты', how='left')

In [10]:
def check_dates(row):
    
    if isinstance(row['Период_действия'], (float)):
        return None
    else:
        return row['Дата приема'].date() in row['Период_действия']

doctor['ПЦР_в_период'] = doctor.apply(check_dates, axis=1)

In [11]:
def to_strf(x):
    if isinstance(x, list):
        return ', '.join([i.strftime('%d.%m.%Y') for i in x])
    
doctor['Период_действия'] = doctor['Период_действия'].apply(to_strf)

In [12]:
doctor.to_excel('Акименко_проверка.xlsx', index=False)